In [1]:
import os
## setting environment variables streaming to be no = False, Yes = True
os.environ["stream"]="No"

## import memory operation manager

In [2]:
from utils import MemoryOps
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank

import os

llm= ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
embed = NVIDIAEmbeddings(model="nvidia/nv-embedqa-mistral-7b-v2",truncate="NONE",)
if os.getenv("stream") :
    stream_flag = os.getenv("stream")
    if stream_flag.lower()=="yes":
        use_streaming = True
    else:
        use_streaming = False
    print("using streaming : ", os.getenv("stream"))
else:
    use_streaming = False
    print("no environment variable set")

memory_ops=MemoryOps(llm,embed,use_streaming )

existing NVIDIA_API_KEY in the environment  nvapi-K
using streaming :  No
using streaming :  No
using streaming in MemoryHandler class ? False


## invoke without streaming 

In [3]:

## setting environment variable to use streaming Yes/No

thread_id=0
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob."
output = await memory_ops.memory_ops_chain.ainvoke(input={"input":query, "config":config})
output

use ainvoke, NO streaming !


"Nice to meet you, Babe! It's not every day that I get to chat with a talking pig, let me tell you! And I'm delighted to hear that you have a best friend like Rob the chicken. What kind of adventures do you and Rob get up to on the farm, I wonder? Do you like to explore the countryside, or do you have a favorite spot to hang out and chat about the latest in farm gossip?"

In [4]:
## now set environment variable before re-import
import os
os.environ["stream"]="Yes"

if os.getenv("stream") :
    stream_flag = os.getenv("stream")
    if stream_flag.lower()=="yes":
        use_streaming = True
    else:
        use_streaming = False
    print("using streaming : ", os.getenv("stream"))
else:
    use_streaming = False
    print("no environment variable set")
memory_ops=MemoryOps(llm,embed,use_streaming )

using streaming :  Yes
using streaming in MemoryHandler class ? True


In [5]:
## invoke with streaming 

In [6]:
thread_id=0
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "I had a fight with Rob, he is no longer my best friend !"
output=""
async for event in memory_ops.memory_ops_chain.astream_events(input={"input":"hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob.", "config":config}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            output += content
            ## comment it out if you dotn want to have large streaming 
            print(output, end=" > ", flush=True)
 

use streaming : True 
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  {"
 streaming output > 
  {"
{"searchfacts >  streaming output > 
  {"facts
{"searchfacts_memory" >  streaming output > 
  {"facts"
 streaming output > " : > 
  {"facts" :
 streaming output > " : [" > 
  {"facts" : ["
 streaming output > 
  {"facts" : ["
 streaming output > " : ["Name > 
  {"facts" : ["Name
 streaming output > 
  {"facts" : ["Name
 streaming output > " : ["Name is > 
  {"facts" : ["Name is
 streaming output > 
  {"facts" : ["Name is
 streaming output > " : ["Name is Babe > 
  {"facts" : ["Name is Babe
 streaming output > 
  {"facts" : ["Name is Babe
{"searchfacts_memory" : ["Name is Babe", >  streaming output > 
  {"facts" : ["Name is Babe",
 streaming output > " : ["Name is Babe", " > 
  {"facts" : ["Name is Babe", "
 streaming output > 
  {"facts" : ["Name is Babe", "
 streaming ou

In [7]:
output

'{"searchfacts_memory" : ["Name is Babe", "Is a pig", "Can talk", "Best friend is a chicken named Rob"]}Nice to meet you, Babe! It\'s not every day that I get to chat with a talking pig, let me tell you! And I\'m delighted to hear that you have a best friend like Rob the chicken. What kind of adventures do you and Rob get up to on the farm, I wonder? Do you like to explore the countryside, or do you have a favorite spot to hang out and chat about the latest in farm gossip?'

In [9]:
if '{' in output:
    index_start=output.index('{')
    index_end=output.index('}')+1
    output=output[index_start:index_end]
    print(output)

{"searchfacts_memory" : ["Name is Babe", "Is a pig", "Can talk", "Best friend is a chicken named Rob"]}
